In [31]:
:dep serde = {version="1.0", features=["derive"]}
:dep serde_cbor = "0.11.2"
:dep serde_json = "1.0"
:dep serde_yaml = "0.8"
:dep serde_qs = "0.9"

In [32]:
use serde::{Deserialize, Serialize};
use std::error::Error;
use std::fs::File;

// Types annotated with `Serialize` can be stored as CBOR.
// To be able to load them again add `Deserialize`.
#[derive(Debug, Serialize, Deserialize)]
struct Mascot {
    name: String,
    species: String,
    year_of_birth: Option<i64>,
}

let ferris = Mascot {
    name: "Ferris".to_owned(),
    species: "crab".to_owned(),
    year_of_birth: Some(2015),
};

# JSON

In [19]:
let mut txt: Vec<u8> = vec![];
println!("{}", serde_json::to_string_pretty(&ferris)?);

{
  "name": "Ferris",
  "species": "crab",
  "year_of_birth": 2015
}


In [21]:
let m: Mascot = serde_json::from_str(r#"
{
  "year_of_birth": 1996,
  "name": "Tux",
  "species": "penguin"
}
"#)?;
m

Mascot { name: "Tux", species: "penguin", year_of_birth: Some(1996) }

# CBOR

In [22]:
let mut v = serde_cbor::to_vec(&ferris)?;
v

[163, 100, 110, 97, 109, 101, 102, 70, 101, 114, 114, 105, 115, 103, 115, 112, 101, 99, 105, 101, 115, 100, 99, 114, 97, 98, 109, 121, 101, 97, 114, 95, 111, 102, 95, 98, 105, 114, 116, 104, 25, 7, 223]

In [23]:
let hex: String = v.iter().map(|b| format!("{:02x}", b)).collect();
println!("Inspect CBOR at https://cbor.me/?bytes={}", &hex);

Inspect CBOR at https://cbor.me/?bytes=a3646e616d6566466572726973677370656369657364637261626d796561725f6f665f62697274681907df


In [8]:
let m: Mascot = serde_cbor::from_slice(&v)?;
m

Mascot { name: "Ferris", species: "crab", year_of_birth: Some(2015) }

In [17]:
let mut v2 = v.clone();
v2[7] += 32;        // lowercase first letter of name
v2[0] = 0b10100010; // change map length to 2
v2.truncate(26);    // truncate third item
let m2: Mascot = serde_cbor::from_slice(&v2)?;
m2

Mascot { name: "ferris", species: "crab", year_of_birth: None }

# Yaml

In [33]:
println!("{}", serde_yaml::to_string(&ferris)?);

---
name: Ferris
species: crab
year_of_birth: 2015



# URL query string

In [34]:
serde_qs::to_string(&ferris)?

"name=Ferris&species=crab&year_of_birth=2015"

In [37]:
let m: Mascot = serde_qs::from_str("name=Tux&species=nice+penguin")?;
m

Mascot { name: "Tux", species: "nice penguin", year_of_birth: None }